In [1]:
import scanpy as sc
import scvelo as scv
import numpy as np
import pandas as pd
import pickle
from veloproj.eval_util import evaluate, summary_scores

dim_size_configs = [(42, 1000), (42, 1500), (42, 2000)]
random_seed_configs = [(19081, 2000), (3490, 2000), (8480, 2000), (11552, 2000), (14590, 2000),
                    (15321, 2000), (22688, 2000), (23419, 2000), (24298, 2000), (24598, 2000)]

settings = {"dentategyrus":{
                'CBDir_input':[('OPC', "OL")],
                'ICVCoh_input':['OPC', "OL"],
                'k_cluster': "clusters",
                'prefix': "ld_dentategyrus"
            },
            "scEU":{
                'CBDir_input':[("3", "1"), ("3", "2")],
                'ICVCoh_input':["1", "2"],
                'k_cluster': "monocle_branch_id",
                'prefix': "ld_scEU"
            },
            "scNT":{
                'CBDir_input':[("0", "15"), ("15", "30"), ("30", "60"), ("60", "120")],
                'ICVCoh_input':["0", "15", "30", "60", "120"],
                'k_cluster': "time",
                'prefix': "ld_scNT"
            },
            "pancreas":{
                'CBDir_input':[("Ngn3 low EP", "Ngn3 high EP"),
                            ("Ngn3 high EP", "Fev+"),
                            ("Fev+", "Alpha"),
                            ("Fev+", "Beta"),
                            ("Fev+", "Delta"),
                            ("Fev+", "Epsilon")],
                'ICVCoh_input':["Ngn3 low EP", "Ngn3 high EP", "Fev+", "Alpha", "Beta", "Delta", "Epsilon"],
                'k_cluster': "clusters",
                'prefix': "ld_pancreas"
            },
            "Erythroid_mouse":{
                'CBDir_input':[("Erythroid1", "Erythroid2"), ('Erythroid2', "Erythroid3")],
                'ICVCoh_input':["Erythroid1", "Erythroid2", "Erythroid3"],
                'k_cluster': "celltype",
                'prefix': "ld_Ery_mouse"
            },
            "Erythroid_human":{
                'CBDir_input':[("Early Erythroid", "Mid  Erythroid"), ('Mid  Erythroid', "Late Erythroid")],
                'ICVCoh_input':["Early Erythroid", "Mid  Erythroid", "Late Erythroid"],
                'k_cluster': "type2",
                'prefix': "ld_Ery_human"
            },
}

def short_metric_name(metric):
    return {"Cross-Boundary Transition Score (A->B)":"CBTrans",
                  "Cross-Boundary Velocity Coherence (A->B)":"CBVCoh",
                  "Cross-Boundary Direction Correctness (A->B)":"CBDir",
                  "In-cluster Confidence":"ICConf",
                  "In-cluster Coherence":"ICVCoh"}[metric]

def score(CBDir_input, ICVCoh_input, k_cluster, file_path):
    adata = sc.read_h5ad(file_path)
    # print(file_path)
    # print(ICVCoh_input[0], type(ICVCoh_input[0]))
    # print(adata.obs[k_cluster] == ICVCoh_input[0])
    # print((adata.obs[k_cluster] == ICVCoh_input[0]).unique())
    # print(type(adata.obs[k_cluster] == ICVCoh_input[0]))
    if file_path.startswith("ld_scNT"):
        adata.obs.time = adata.obs.time.astype('str')
        # print(adata.obs.time.unique())
        # sel = adata.obs[k_cluster].values == "0"
        # print(np.sum(sel))
        # print(adata.uns['neighbors']['indices'].shape)
        # print(adata.uns['neighbors']['indices'][sel])  
    
    scv.tl.velocity_confidence(adata, vkey='velocity')
    scores = evaluate(adata, CBDir_input, k_cluster, "velocity", verbose=False)
    CBDir_all_scores = scores['Cross-Boundary Direction Correctness (A->B)']
    ICVCoh_all_scores = scores['In-cluster Coherence']
    CBDir_score_by_group, _ = summary_scores(CBDir_all_scores)
    if len(CBDir_score_by_group) == 0 or np.isnan(_):
        print(CBDir_input, CBDir_score_by_group, _)
        print([CBDir_score_by_group[g] if g in CBDir_score_by_group else 0 for g in CBDir_input])
        return None, None
    CBDir_score = np.mean([CBDir_score_by_group[g] if g in CBDir_score_by_group else 0 for g in CBDir_input])
    ICVCoh_score_by_group, _ = summary_scores(ICVCoh_all_scores)
    ICVCoh_score = np.mean([ICVCoh_score_by_group[g] if g in ICVCoh_score_by_group else 0 for g in ICVCoh_input])
    return CBDir_score, ICVCoh_score

In [2]:
dim_size_scores = {}
errors = {}
for dataset in settings:
    dim_size_scores[dataset] = {"CBDir":[], "ICVCoh":[]}
    for seed, dim in dim_size_configs:
        prefix = settings[dataset]['prefix']
        CBDir_input = settings[dataset]['CBDir_input']
        ICVCoh_input = settings[dataset]['ICVCoh_input']
        k_cluster = settings[dataset]['k_cluster']

        file_path = f"{prefix}_{seed}_{dim}.h5"
        cbdir, icvcoh = score(CBDir_input, ICVCoh_input, k_cluster, file_path)
        if cbdir is None or icvcoh is None:
            print(file_path, " has unkown values")
            adata = sc.read_h5ad(file_path)
            errors[file_path] = adata
            
        
        dim_size_scores[dataset]['CBDir'].append(cbdir)
        dim_size_scores[dataset]['ICVCoh'].append(icvcoh)
        print("-"*25)
print(dim_size_scores)

--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
-------------------------
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
-------------------------
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
-------------------------
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
-------------------------
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
-------------------------
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
-------------------------
--> added 

In [3]:
# adata = sc.read_h5ad("ld_scNT_42_500.h5")
# adata.obs.time.astype("str") == settings['scNT']['ICVCoh_input'][0]
!which python

/data/users/cqiao/conda_envs/bin/python


In [4]:
random_seed_scores = {}
for dataset in settings:
    random_seed_scores[dataset] = {"CBDir":[], "ICVCoh":[]}
    for seed, dim in random_seed_configs:
        prefix = settings[dataset]['prefix']
        CBDir_input = settings[dataset]['CBDir_input']
        ICVCoh_input = settings[dataset]['ICVCoh_input']
        k_cluster = settings[dataset]['k_cluster']

        file_path = f"{prefix}_{seed}_{dim}.h5"
        cbdir, icvcoh = score(CBDir_input, ICVCoh_input, k_cluster, file_path)
        random_seed_scores[dataset]['CBDir'].append(cbdir)
        random_seed_scores[dataset]['ICVCoh'].append(icvcoh)
print(random_seed_scores)

--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
--> added 'velocity_length'

In [9]:
with open("scores.pkl",'wb') as ofile:
    pickle.dump((random_seed_scores, dim_size_scores), ofile)

In [3]:
with open("scores.pkl", 'rb') as ifile:
    random_seed_scores, dim_size_scores = pickle.load(ifile)

In [15]:

dim_table_values = []
k_coh = 'ICVCoh'
k_cb = 'CBDir'
columns = [f"{k_coh}_{dim}" for dim in [1000, 1500, 2000]] + [f"{k_cb}_{dim}" for dim in [1000, 1500, 2000]]
index = list(dim_size_scores.keys())
df_dim_scores = pd.DataFrame(index=index, columns=columns)
for dataset, score_dict in dim_size_scores.items():
    df_dim_scores.loc[dataset] = np.round(score_dict[k_coh] + score_dict[k_cb], 4)
df_dim_scores.loc['scEU'][3:] = 'NA'
df_dim_scores


,ICVCoh_1000,ICVCoh_1500,ICVCoh_2000,CBDir_1000,CBDir_1500,CBDir_2000
dentategyrus,1.0,1.0,1.0,0.8108,0.9461,0.8947
scEU,0.9993,0.9996,0.9997,NA,NA,NA
scNT,0.9987,1.0,0.9999,0.4172,0.4337,0.4294
pancreas,0.9975,0.9998,0.9998,0.5508,0.5711,0.5432
Erythroid_mouse,0.9998,0.9998,0.9998,0.6262,0.626,0.6234
Erythroid_human,0.9974,0.9991,0.9987,0.3442,0.3689,0.4152


In [24]:
random_table_values = []
k_coh = 'ICVCoh'
k_cb = 'CBDir'
columns = list(random_seed_scores.keys())
index = [k_coh, k_cb]
df_random_scores = pd.DataFrame(index=index, columns=columns)
for dataset, score_dict in random_seed_scores.items():
    for k in [k_coh, k_cb]:
        df_random_scores.loc[k, dataset] = f"{np.mean(score_dict[k]):.2f}±{np.std(score_dict[k]):.2f}"
df_random_scores.loc[k_cb]['scEU'] = 'NA'
df_random_scores

,dentategyrus,scEU,scNT,pancreas,Erythroid_mouse,Erythroid_human
ICVCoh,1.00±0.00,1.00±0.00,1.00±0.00,1.00±0.00,1.00±0.00,1.00±0.00
CBDir,0.84±0.09,NA,0.39±0.02,0.53±0.04,0.63±0.01,0.43±0.01
